## Imports

In [41]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library sqlite3

In [42]:
connection = sqlite3.connect('../data/checking-logs.sqlite.sqlite copy')
connection

## create a new table datamart in the database by joining the tables pageviews and checker using only one query:
#### 1) the table should have the following columns: uid, labname, first_commit_ts, first_view_ts
#### 2) first_commit_ts is just a new name of the column timestamp from the checker table, it shows the first commit from a particular lab and from a particular user
#### 3) first_view_ts is the first visit of a user to the table pageviews, timestamp when a user visited the newsfeed
#### 4) status = ’ready’ should still be a filter
#### 5) numTrials = 1 should still be a filter
#### 6) labnames should still be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’
#### 7) the table should contain only the users (uids with user_*) and not the admins
#### 8) first_commit_ts and first_view_ts should be parsed as datetime64[ns]

In [43]:
query = '''
SELECT checker.uid, checker.labname, checker.timestamp AS first_commit_ts, pageviews.datetime AS first_view_ts
FROM checker LEFT JOIN pageviews ON checker.uid = pageviews.uid
WHERE status = 'ready' AND numTrials = 1 AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
AND checker.uid LIKE 'user_%' AND (pageviews.datetime = (SELECT MIN(pageviews.datetime)
                                                        FROM pageviews
                                                        WHERE uid = checker.uid) OR pageviews.datetime IS NULL)
'''

datamart = pd.io.sql.read_sql(query, connection, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


In [44]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


## Using Pandas methods, create two dataframes: test and control:
#### 1) test should have the users that have the values in first_view_ts
#### 2) control should have the users that have missing values in first_view_ts
#### 3) replace the missing values in the control with the average first_view_ts of the test users, we will use this value for the future analysis
#### 4) save both tables into the database, you will use them in the next exercises

In [45]:
test = datamart[datamart['first_view_ts'].notna()].reset_index(drop=True) # 1)
test.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447


In [46]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.0+ KB


In [47]:
control = datamart[datamart['first_view_ts'].isna()].reset_index(drop=True) # 2)
control.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_2,laba04,2020-04-18 13:42:35.482008,NaT
4,user_2,laba04s,2020-04-18 13:51:22.291271,NaT


In [48]:
control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.7+ KB


In [49]:
control.fillna(test['first_view_ts'].mean(), inplace=True) # 3)
control.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552
3,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783552
4,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783552


In [50]:
control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.7+ KB


In [58]:
test.to_sql('test', connection) # 4)
control.to_sql('control', connection)

ProgrammingError: Cannot operate on a closed database.

## close the connection

In [59]:
connection.close()